In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import GBTClassifier

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark gb ML example") \
    .getOrCreate()

In [ ]:
# Load and parse the data file, converting it to a DataFrame.
data = spark.read.format("com.databricks.spark.csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("adult5.csv")



In [ ]:
data.show()

In [ ]:
categoricalColumns = ["workclass", "occupation"]

In [ ]:
stages = []
for categoricalCol in categoricalColumns:
	stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
	#.fit(ad_data)
	#df_numeric = stringIndexer.transform(ad_data)
	#df_numeric.repartition(1).repartition(1).write.csv('indexer')
	#print df_numeric.select('workclass','workclassIndex').show(5)
	#In the above line for example, it takes workclass string and concatinates with the address("Index")
	encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
	#print encoder.outputCol
	stages += [stringIndexer, encoder]

In [ ]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
label_stringIdx = StringIndexer(inputCol = "income", outputCol = "labelindex")

In [ ]:
stages += [label_stringIdx]

In [ ]:
numericCols = ["age", "hours_per_week"]

In [ ]:
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) 

In [ ]:
assemblerInputs=list(assemblerInputs) + numericCols

In [ ]:
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

In [ ]:
stages += [assembler]

In [ ]:
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.

In [ ]:
pipeline = Pipeline(stages=stages)

In [ ]:
pipelineModel = pipeline.fit(data)
dataset = pipelineModel.transform(data)

In [ ]:
cols = data.columns

In [ ]:
selectedcols = ["labelindex", "features"] + cols
dataset = dataset.select(selectedcols)

In [ ]:
from pyspark.ml.feature import VectorIndexer

In [ ]:
dataset.printSchema()
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(dataset)
print(featureIndexer)



In [ ]:
featuredf=featureIndexer.transform(dataset)

In [ ]:
featuredf.show()

In [ ]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = featuredf.randomSplit([0.7, 0.3], seed = 100)

In [ ]:
trainingData.show()

In [ ]:
testData.show()

In [ ]:
# Train a GBT model.
gbt =GBTClassifier(labelCol="labelindex", featuresCol="indexedFeatures", maxIter=10)

In [ ]:
# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

In [ ]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [ ]:
# Make predictions.
predictions = model.transform(testData)

predictions.printSchema()

In [ ]:
trainingData.show()

In [ ]:
# Select example rows to display.
predictions.select("prediction", "labelindex", "features","probability","rawPrediction").show(5)

In [ ]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="labelindex", predictionCol="prediction", metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(predictions)

In [ ]:
print("Test Error = %g" % (1.0 - accuracy))

In [ ]:
gbtModel = model.stages[1]

In [ ]:
print(gbtModel)# summary only